In [1]:
import neat
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import networkx as nx

import reservoirpy as rpy
rpy.verbosity(0)

from reservoirpy.nodes import Reservoir
from reservoirpy import nodes, datasets

In [2]:
# Define other required parameters
population_size = 50
n_generations = 50
genome_length = 100
n_timesteps = 2000

# Initialize your dataset
#X = datasets.mackey_glass(n_timesteps=n_timesteps, sample_len=2000)
#Y = np.roll(X, -1)

X, Y = datasets.mackey_glass(n_timesteps=n_timesteps, sample_len=2000, tau=17), np.roll(datasets.mackey_glass(n_timesteps=n_timesteps, sample_len=2000, tau=17), -1)

# Split the dataset
# train_end = int(len(X) * 0.7)
# test_start = train_end + 1
# X_train, Y_train = X[:train_end], Y[:train_end]
# X_test, Y_test = X[test_start:], Y[test_start:]

train_end, test_start = int(len(X) * 0.7), int(len(X) * 0.7) + 1
X_train, Y_train, X_test, Y_test = X[:train_end], Y[:train_end], X[test_start:], Y[test_start:]

# Ensure X and Y are correctly shaped for ReservoirPy
X_train = X_train.reshape(-1, 1)
Y_train = Y_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)
Y_test = Y_test.reshape(-1, 1)

In [3]:
def plot_performance(rmse_scores, mae_scores, mse_scores): 
    plt.figure(figsize=(12, 6))
    plt.plot(rmse_scores, marker='o', linestyle='-', color='b', label='RMSE')
    plt.title('Performance Scores Over Generations')
    plt.xlabel('Generation')
    plt.ylabel('RMSE Score')
    plt.legend()
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(mse_scores, marker='^', linestyle='-', color='g', label='MSE')
    plt.title('Performance Scores Over Generations')
    plt.xlabel('Generation')
    plt.ylabel('MSE Score')
    plt.legend()
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(mae_scores, marker='x', linestyle='-', color='r', label='MAE')
    plt.title('Performance Scores Over Generations')
    plt.xlabel('Generation')
    plt.ylabel('MAE Score')
    plt.legend()
    plt.grid(True)
    plt.show()

In [5]:

mse_values, mae_values, rmse_values = [], [], []

# Define your evaluation function using the loaded dataset
def eval_genomes_old(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        #predictions = np.array([net.activate(xi) for xi in X_train])
        predictions = []
        for xi in X_train:  # X_train should have a shape (n_samples, 10)
            output = net.activate(xi)  # xi needs to be an iterable of length 10
            predictions.append(output)
        predictions = np.array(predictions)
        mse = mean_squared_error(Y_train, predictions)
        mae = mean_absolute_error(Y_train, predictions)
        rmse = np.sqrt(mse)
                
        # Combining fitness metrics, you can customize this part
        genome.fitness = -mse  # NEAT works with a maximization problem
    
    mse_values.append(np.min(mse))
    mae_values.append(np.min(mae))
    rmse_values.append(np.min(rmse))

def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        
        # Setup reservoir
        reservoir = Reservoir(100, lr=0.3, input_scaling=0.5)
        
        # Process inputs through the reservoir
        reservoir_output = reservoir.run(X_train)

        # Flatten the output for NEAT's network
        enhanced_X_train = reservoir_output.reshape(reservoir_output.shape[0], -1)

        predictions = np.array([net.activate(xi) for xi in enhanced_X_train])
        
        mse = mean_squared_error(Y_train, predictions)
        mae = mean_absolute_error(Y_train, predictions)
        rmse = np.sqrt(mse)
        genome.fitness = -mse  # In NEAT, higher fitness is better, hence the negation
        
    mse_values.append(np.min(mse))
    mae_values.append(np.min(mae))
    rmse_values.append(np.min(rmse))
    
# Load the configuration
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, 'config_neat.txt')

# Create the population, which is the top-level object for a NEAT run.
p = neat.Population(config)

# Add a stdout reporter to show progress in the terminal.
p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)
p.add_reporter(neat.Checkpointer(5))

# Run the experiment, pass eval_genomes function and number of generations
winner = p.run(eval_genomes, n_generations)

# Display the winning genome
print('\nBest genome:\n{!s}'.format(winner))

# Plotting the performance graph
plot_performance(rmse_values, mae_values, mse_values)



 ****** Running generation 0 ****** 



RuntimeError: Expected 1 inputs, got 100